# data 가져오기

In [1]:
import pandas as pd
import numpy as np

In [2]:
datapath = "./data/winequalityN.csv"

data_sets = pd.read_csv(datapath, delimiter = ',')

data_sets.dropna(inplace = True)

x_data = data_sets.drop(columns = ["type", "quality"],axis = 1)
y_data = data_sets["quality"]

x_data = x_data.dropna(axis = 0)
y_data = y_data.dropna(axis = 0)

In [3]:
y_data = np.where(data_sets['quality'] > 5,1.0,0.0)
x_data = x_data.values.tolist()

In [4]:
x_data = np.array(x_data)
y_data = np.array(y_data)

# SVM

In [5]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC

# train, val, test 나누기

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data,test_size = 0.2, random_state=0)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2,random_state=0)

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(4136, 11)
(4136,)
(1034, 11)
(1034,)
(1293, 11)
(1293,)


# 모델 선택 (교차 검증)

In [23]:
def model_selection(x_data, y_data):
    kernels = ['linear','poly','rbf','sigmoid']
    svm_model = []
    scores = []
    
    for i in kernels:
        temp_model = make_pipeline(StandardScaler(), svm.SVR(C = 1.0, epsilon = 0.1, kernel = i))
        scores.append(cross_val_score(temp_model, x_data, y_data).mean())
    
    for i,j in enumerate(scores):
        print("model : {}, score : {}".format(kernels[i], j))
        
model_selection(x_data,y_data)

model : linear, score : 0.0655945814786
model : poly, score : -10.9157121379
model : rbf, score : 0.105497203972
model : sigmoid, score : -5662.44913637


# train

In [34]:
def get_predictions(model, x_data):
    predictions = model.predict(x_data)
    predictions = np.around(predictions)
    
    return predictions

def model(x_train, y_train):
    svm_model = make_pipeline(StandardScaler(), svm.SVR(C = 1.0, epsilon = 0.3, kernel = 'rbf'))

    svm_model.fit(x_train, y_train)
    
    predictions = get_predictions(svm_model, x_train)
    
    return svm_model, predictions

m1, p = model(x_train, y_train)

# Val 평가

In [35]:
score = m1.score(x_val, y_val)

print("SVM 의 교차검증 점수 : {}".format(score))

SVM 의 교차검증 점수 : 0.303846601548


# Val train

In [36]:
m1.fit(x_val, y_val)

Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svr', SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.3,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False))])

# train 평가

In [37]:
print("모델 평가 : {}".format(m1.score(x_train, y_train)))
print("Train set 정확도: {:.2f}".format(accuracy_score(y_train, p)))

모델 평가 : 0.257953228798
Train set 정확도: 0.82


# test로 정확도 측정

In [38]:
print("Test set 정확도: {:.2f}".format(accuracy_score(y_test, get_predictions(m1, x_test))))

Test set 정확도: 0.74
